This notebook demonstrates how chaining `TextModifier`s with `gridsearch.iter_strings` can help solve grid-based puzzles.

Here's a hypothetical puzzle (inspired by a puzzle in an old issue of Panda Magazine, which I solved by hand but then used as a test case when writing the grid searching functions):

------
# By Any Other Name
*You're always forgetting the terms for things.*
```
y q a y y c o z f y x u j h a  
c i t c r a d n a w i n k f t  
s z r x i n x m r y b s c r v  
l l w r q s a x p o b p a e i  
l l j e d n n u f k r e t s k  
u w k g s g z a y z e i t c q  
o b t g l v h s b p a b l c m  
s d o a o s m o m u l l e i h  
m p m d v c y p s l e e n d d  
k g o d q p f e l z g o s e f  
e g f n f v g k s o t r l n s  
y x w n g a y s j v j p l t w  
y e s o e q d n w f f r a e l  
i o w l o o s c t e h g j x a  
z u a c l o b a u k u v m y c
```
* Batavia (7)
* Black Panther Homeland (7)
* "Good luck!" (5 1 3)
* Imre Lipschitz (7)
* Indescribable (11)
* Nobel Prize-winner Shinya (8)
* Owl Parrots (7)
* Random Mishap (5 8)
* Slow Drip (7)
* Sudden Strike (5 6)
* Ty and Tandy (5 3 6)
------

So you poke at this for a bit, and you see answers for a few of these clues - you happen to know that Imre Lipschitz changed his last name to Lakatos, that the Indonesian capital of Batavia is now called Jakarta, and that Ty Johnson and Tandy Bowen are the alter-egos of Marvel superhero duo Cloak And Dagger.

You don't see any of these in the grid, but you do see "clonnddagger" running up the fourth column, so you guess (correctly) that every answer contains the trigram "aka", but that trigram has been replaced by another letter before putting them into the actual grid.

You could now find the answers for the other clues, hunt through the grid for them, and work from there. But maybe you're eager to move on to other puzzles, and you'd rather have Python do the grunt work of searching the grid for you!

So first, you parse the grid into a numpy array, using `puztool.parse_grid`, or the shorter alias `puztool.pg`:

In [1]:
from puztool import pg
grid = pg('''
y q a y y c o z f y x u j h a  
c i t c r a d n a w i n k f t  
s z r x i n x m r y b s c r v  
l l w r q s a x p o b p a e i  
l l j e d n n u f k r e t s k  
u w k g s g z a y z e i t c q  
o b t g l v h s b p a b l c m  
s d o a o s m o m u l l e i h  
m p m d v c y p t l e e n d d  
k g o d q p f e l z g o s e f  
e g f n f v g k s o t r l n s  
y x w n g a y s j v j p l t w  
y e s o e q d n w f f r a e l  
i o w l o o s c t e h g j x a  
z u a c l o b a u k u v m y c
''')

You know from the enumerations that the shortest answers are 7 letters long, which means the shortest strings you care about in this grid will be 5 characters long. So you can use `iter_strings` to find all 5+-character strings in the grid. The returned values will be `Result` objects where `.val` is the found string and `.provenance` is a tuple of `(start, end)` showing the coordinates of the first and last letters of that word in the grid. There are 5920 strings here, so let's just see a few of them:

In [2]:
from puztool.gridsearch import iter_strings
for i,s in enumerate(iter_strings(grid, len=(5,None))):
    if not i%1000: # only show every 1000th because there are a LOT
        print(s.val, s.provenance)

yqayy ((0, 0), (0, 4))
xnixr ((2, 6), (2, 2))
uwkgsgzayzeit ((5, 0), (5, 12))
oypfn ((7, 7), (11, 3))
egfnfvgksotrl ((10, 0), (10, 12))
fvozlupzkoywy ((12, 9), (0, 9))


Now we want to look at all strings that can be produced by taking a string from this and replacing a single character with `'aka'`. We can write this as a `puztool.TextModifier` - a function that takes a `Result` object and returns an iterable of `Result` objects, and automatically knows how to do things like chain with other modifiers. Here's that modifier:

In [3]:
from puztool import FnModifier
@FnModifier
def add_aka(result):
    s = result.val
    for i in range(len(s)):
        yield result.extend(s[:i]+'aka'+s[i+1:], (s, s[i]))

`Result.extend(val, provenance)` returns a new `Result` with the new value but with the new provenance *appended to* the old provenance. In this case, our `add_aka` processor is adding both the original string and which letter was replaced to the provenance chain so we can refer to it later. Thus, a single result looks like this:

In [4]:
result = next(add_aka(iter_strings(grid, len=(5,None))))
print(result.val, result.provenance)

akaqayy ((0, 0), (0, 4), ('yqayy', 'y'))


Finally, we can restrict the output to words or phrases in a wordlist using the `puztool.In` modifier as a filter. `puztool.lists.<name>` fetches a `WordList` object derived from `data/wordlists/<name>.txt`; this library doesn't ship with any word lists because they're all enormous, but I use a list stored as `npl.txt` that is just the NPL's `allwords.txt` with punctuation, spaces, etc. stripped out. So we can join `add_aka` to a filter with:

In [5]:
from puztool import In, lists
check = add_aka | In(lists.npl)

Now we can run this on the full list of strings in the grid. Since the return values are `Result` objects with same-length provenances, it's helpful to unpack them into a pandas DataFrame so that they render nicely in this notebook:

In [6]:
import pandas as pd
results = iter_strings(grid, len=(5, None) ) | check.l()
results = pd.DataFrame([r.val, *r.provenance] for r in results)
results

,0,1,2,3
0,yamanaka,"(0, 9)","(5, 4)","(yamans, s)"
1,unspeakable,"(0, 11)","(8, 11)","(unspeible, i)"
2,wakanda,"(1, 9)","(1, 5)","(wanda, a)"
3,freakaccident,"(1, 13)","(11, 13)","(fresccident, s)"
4,speakable,"(2, 11)","(8, 11)","(speible, i)"
5,breakaleg,"(3, 10)","(9, 10)","(brealeg, a)"
6,jakarta,"(4, 2)","(0, 2)","(jwrta, w)"
7,sneakattack,"(9, 12)","(1, 12)","(snelttack, l)"
8,kakapos,"(10, 7)","(6, 7)","(kepos, e)"
9,lakatos,"(10, 12)","(10, 8)","(lrtos, r)"


The above chart shows all the words or phrases found in the grid, followed by the coordinates where the word was located, then followed by the actual string that was in the grid and the letter that needs to be replaced by `'aka'` to yield the answer to the clue. Obviously there are a few extras - "speakable" and "leakage" aren't answers, they're just substrings of the answers "unspeakable" and "leakages" that happen to be words themselves.

But you can pick out the correct answers and their locations from this chart easily, or, if you think of it, you might try resorting these by the answers and looking at the extra letters:

In [7]:
results.sort_values(0)

,0,1,2,3
5,breakaleg,"(3, 10)","(9, 10)","(brealeg, a)"
11,cloakanddagger,"(14, 3)","(3, 3)","(clonnddagger, n)"
3,freakaccident,"(1, 13)","(11, 13)","(fresccident, s)"
6,jakarta,"(4, 2)","(0, 2)","(jwrta, w)"
8,kakapos,"(10, 7)","(6, 7)","(kepos, e)"
9,lakatos,"(10, 12)","(10, 8)","(lrtos, r)"
10,leakage,"(13, 3)","(9, 7)","(leage, a)"
7,sneakattack,"(9, 12)","(1, 12)","(snelttack, l)"
4,speakable,"(2, 11)","(8, 11)","(speible, i)"
1,unspeakable,"(0, 11)","(8, 11)","(unspeible, i)"


Even with the extra words in the `DataFrame`, it's easy to read the text `answeralias` in those extra letters. The answer to the puzzle, therefore, is **alias**.